# Reinforcement Learning Tutorial

Some Common Terms:

Agent

Environment

Actions, Rewards, Observations



### 1. Interacting with the Gym API

In [10]:
import gym

In [11]:
# Create Enviroment 
env = gym.make('CartPole-v0')

Each environment comes with certain import methods/attributes:

action_space

observation_space

reset() : returns init state and also resets the environment

step()

render()


In [12]:
env.reset()

array([-0.04485218, -0.0387376 , -0.02056453, -0.00626859], dtype=float32)

In [13]:
env.action_space

Discrete(2)

In [14]:
env.action_space.n

2

In [15]:
env.observation_space.shape[0]

4

In [8]:
env.reset()
for t in range(1000):
    random_action = env.action_space.sample()
    env.step(random_action) #randomly move left or right
    env.render()
env.close()
#env.reset()

c:\users\siddharth\python38\lib\site-packages\gym\envs\classic_control\cartpole.py:150: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  logger.warn(


### 2. Playing Games with a Random Strategy

Game Episode

Step() Function in More Detail

Game Over?

In [16]:
for e in range(20): #Episode
    #Play 20 episodes 
    observation = env.reset()
    for t in range(50):
        env.render()
        action = env.action_space.sample()
        observation,reward,done,other_info = env.step(action)
        
        if done:
            # Game Episode is over
            print("Game Episode :{}/{} High Score :{}".format(e,20,t))
            break

env.close()
print("All 20 episodes over!")

Game Episode :0/20 High Score :14
Game Episode :1/20 High Score :13
Game Episode :2/20 High Score :28
Game Episode :3/20 High Score :17
Game Episode :4/20 High Score :14
Game Episode :5/20 High Score :22
Game Episode :6/20 High Score :14
Game Episode :7/20 High Score :9
Game Episode :8/20 High Score :12
Game Episode :9/20 High Score :17
Game Episode :10/20 High Score :12
Game Episode :11/20 High Score :17
Game Episode :12/20 High Score :9
Game Episode :13/20 High Score :12
Game Episode :14/20 High Score :22
Game Episode :15/20 High Score :27
Game Episode :16/20 High Score :15
Game Episode :17/20 High Score :22
Game Episode :18/20 High Score :16
Game Episode :19/20 High Score :10
All 20 episodes over!


### 3. Q-Learning
Designing an AI Agent

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import os
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import random

In [18]:
class Agent:
    def __init__(self,state_size,action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95 #Discount Factor
        self.epsilon = 1.0 # Exploration Rate: How much to act randomly, 
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.learning_rate = 0.001 
        self.model = self._create_model()
        
    
    def _create_model(self):
        #Neural Network To Approximate Q-Value function
        model = Sequential()
        model.add(Dense(24,input_dim=self.state_size,activation='relu')) #1st Hidden Layer
        model.add(Dense(24,activation='relu')) #2nd Hidden Layer
        model.add(Dense(self.action_size,activation='linear'))
        model.compile(loss='mse',optimizer=Adam(learning_rate=self.learning_rate))
        return model
    
    def remember(self,state,action,reward,next_state,done):
        self.memory.append((state,action,reward,next_state,done)) #remembering previous experiences
        
    def act(self,state):
        # Exploration vs Exploitation
        if np.random.rand()<=self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state) # predict reward value based upon current state
        return np.argmax(act_values[0]) #Left or Right
    
    def train(self,batch_size=32): #method that trains NN with experiences sampled from memory
        minibatch = random.sample(self.memory,batch_size)
        for state,action,reward,next_state,done in minibatch:
            
            if not done: #boolean 
                target = reward + self.gamma*np.amax(self.model.predict(next_state)[0])
            else:
                target = reward
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state,target_f,epochs=1,verbose=0) #single epoch, x =state, y = target_f, loss--> target_f - 
            
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    def load(self,name):
        self.model.load_weights(name)
    def save(self,name):
        self.model.save_weights(name)
            

### Training the DQN Agent (Deep Q-Learner)

In [19]:
n_episodes = 1000
output_dir = "cartpole_model/"

In [20]:
agent = Agent(state_size=4,action_size=2)
done = False
state_size = 4
action_size =2
batch_size = 32

In [21]:
agent = Agent(state_size, action_size) # initialise agent
done = False
for e in range(n_episodes):
    state = env.reset()
    state = np.reshape(state,[1,state_size])
    
    for time in range(5000):
        env.render()
        action = agent.act(state) #action is 0 or 1
        next_state,reward,done,other_info = env.step(action) 
        reward = reward if not done else -10
        next_state = np.reshape(next_state,[1,state_size])
        agent.remember(state,action,reward,next_state,done)
        state = next_state
        
        if done:
            print("Game Episode :{}/{}, High Score:{},Exploration Rate:{:.2}".format(e,n_episodes,time,agent.epsilon))
            break
            
    if len(agent.memory)>batch_size:
        agent.train(batch_size)
    
    if e%50==0:
        agent.save(output_dir+"weights_"+'{:04d}'.format(e)+".hdf5")
        
env.close()

Game Episode :0/1000, High Score:12,Exploration Rate:1.0


FileNotFoundError: [Errno 2] Unable to create file (unable to open file: name = 'cartpole_model/weights_0000.hdf5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 302)